## Задача: классификация изображений из датасетап CIFAR-10. Проект включает в себя обучение модели сверточной нейронной сети (CNN) на PyTorch и создание интерактивного веб-приложения на Streamlit, позволяющего пользователю загружать изображения для классификации.

### Импорт библиотек

In [5]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

### Трансформирование изображений в тензоры и нормализация.

In [18]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

### Загрузка данных, разделение на train/test.

In [19]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

In [20]:
image, label = trainset[0]

# image

In [21]:
image.size()

torch.Size([3, 32, 32])

In [22]:
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

### Создание архитектуры CNN.

In [26]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 12, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(12, 24, 5)
        self.fc1 = nn.Linear(24 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

### Выбор функции потерь и оптимизатора.

In [27]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.001, momentum = 0.9)

### Тренировка модели

In [28]:
for epoch in range(30):
    print(f'training {epoch} epoch')
    
    running_loss = 0.0
    
    for i, data in enumerate(trainloader):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)

        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Loss: {running_loss / len(trainloader):.4f}')

training 0 epoch
Loss: 1.6538
training 1 epoch
Loss: 1.2443
training 2 epoch
Loss: 1.0786
training 3 epoch
Loss: 0.9709
training 4 epoch
Loss: 0.8825
training 5 epoch
Loss: 0.8179
training 6 epoch
Loss: 0.7653
training 7 epoch
Loss: 0.7185
training 8 epoch
Loss: 0.6741
training 9 epoch
Loss: 0.6454
training 10 epoch
Loss: 0.6138
training 11 epoch
Loss: 0.5922
training 12 epoch
Loss: 0.5593
training 13 epoch
Loss: 0.5412
training 14 epoch
Loss: 0.5244
training 15 epoch
Loss: 0.5123
training 16 epoch
Loss: 0.4917
training 17 epoch
Loss: 0.4792
training 18 epoch
Loss: 0.4712
training 19 epoch
Loss: 0.4604
training 20 epoch
Loss: 0.4552
training 21 epoch
Loss: 0.4428
training 22 epoch
Loss: 0.4360
training 23 epoch
Loss: 0.4296
training 24 epoch
Loss: 0.4264
training 25 epoch
Loss: 0.4242
training 26 epoch
Loss: 0.4144
training 27 epoch
Loss: 0.4195
training 28 epoch
Loss: 0.4152
training 29 epoch
Loss: 0.4073


### Ассессмент перформанса модели

In [31]:
correct = 0
total = 0

net.eval()

with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy: {accuracy}%')

Accuracy: 62.86%


### Сохранение модели.

In [32]:
torch.save(net.state_dict(), 'cifar_net.pth')